# Clasificación de imágenes simples - SVM

### Estudiante: Rebeca Justiniano Saravia

Dataset de imagenes de frutas (manzana/banana/naranja)

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from torchvision import datasets, transforms

## 1. Extraer características simples (color promedio, tamaño, relación de aspecto)

In [21]:
# Cargar dataset SIN normalización (para extraer características)
transform_simple = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()  # Solo convertir a tensor [0,1]
])

In [22]:
dataset = datasets.ImageFolder('Datasets/frutas', transform=transform_simple)

In [23]:
def extraer_caracteristicas(imagen_tensor):
    """
    Extrae características simples de una imagen tensor
    imagen_tensor: tensor de forma (C, H, W) con valores [0,1]
    """
    # Convertir a numpy y formato HWC
    img_np = imagen_tensor.permute(1, 2, 0).numpy()
    
    # 1. Color promedio (RGB)
    color_promedio_r = img_np[:, :, 0].mean()
    color_promedio_g = img_np[:, :, 1].mean()
    color_promedio_b = img_np[:, :, 2].mean()
    
    # 2. Desviación estándar de color (variación de color)
    std_r = img_np[:, :, 0].std()
    std_g = img_np[:, :, 1].std()
    std_b = img_np[:, :, 2].std()
    
    # 3. Tamaño (alto y ancho)
    altura, ancho = img_np.shape[:2]
    
    # 4. Relación de aspecto
    relacion_aspecto = ancho / altura
    
    # 5. Brillo promedio
    brillo = img_np.mean()
    
    caracteristicas = {
        'color_promedio_r': color_promedio_r,
        'color_promedio_g': color_promedio_g,
        'color_promedio_b': color_promedio_b,
        'std_r': std_r,
        'std_g': std_g,
        'std_b': std_b,
        'altura': altura,
        'ancho': ancho,
        'relacion_aspecto': relacion_aspecto,
        'brillo': brillo
    }
    
    return caracteristicas

In [24]:
# Extraer características de todo el dataset
todas_caracteristicas = []
todas_etiquetas = []

for imagen, etiqueta in dataset:
    features = extraer_caracteristicas(imagen)
    todas_caracteristicas.append(features)
    todas_etiquetas.append(etiqueta)

In [25]:
# Convertir a arrays de numpy para análisis
df = pd.DataFrame(todas_caracteristicas)
df['clase'] = [dataset.classes[label] for label in todas_etiquetas]
df['etiqueta'] = todas_etiquetas

print("Características extraídas:")
print(df.head())
print("\nEstadísticas por clase:")
print(df.groupby('clase').mean())

Características extraídas:
   color_promedio_r  color_promedio_g  color_promedio_b     std_r     std_g  \
0          0.763332          0.536880          0.521778  0.238508  0.414771   
1          0.910967          0.825253          0.826278  0.188302  0.323605   
2          0.878256          0.647770          0.589854  0.183723  0.341885   
3          0.674970          0.568836          0.375606  0.185004  0.251380   
4          0.794495          0.612212          0.630183  0.288324  0.397992   

      std_b  altura  ancho  relacion_aspecto    brillo        clase  etiqueta  
0  0.415831     224    224               1.0  0.607330  apple fruit         0  
1  0.318342     224    224               1.0  0.854166  apple fruit         0  
2  0.370680     224    224               1.0  0.705293  apple fruit         0  
3  0.247876     224    224               1.0  0.539804  apple fruit         0  
4  0.385498     224    224               1.0  0.678963  apple fruit         0  

Estadísticas por 

## 2. Entrenar un SVM y árbol de decisión

### SVM

In [26]:
# 1. Preparar los datos
# Seleccionar solo las columnas de características (sin 'clase' y 'etiqueta')
columnas_features = ['color_promedio_r', 'color_promedio_g', 'color_promedio_b',
                     'std_r', 'std_g', 'std_b', 'altura', 'ancho', 
                     'relacion_aspecto', 'brillo']

X = df[columnas_features].values
y = df['etiqueta'].values

print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

Forma de X: (120, 10)
Forma de y: (120,)


In [27]:
# 2. Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nEntrenamiento: {X_train.shape[0]} muestras")
print(f"Prueba: {X_test.shape[0]} muestras")


Entrenamiento: 96 muestras
Prueba: 24 muestras


In [28]:
# 3. Normalizar las características (MUY IMPORTANTE para SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [29]:
# 4. Entrenar el modelo SVM
svm_model = SVC(
    kernel='rbf',  # Kernel RBF (Radial Basis Function)
    C=1.0,         # Parámetro de regularización
    gamma='scale', # Parámetro del kernel
    random_state=42
)

svm_model.fit(X_train_scaled, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [30]:
# 5. Hacer predicciones
y_pred = svm_model.predict(X_test_scaled)

### Arbol de decisión

In [31]:
dt_model = DecisionTreeClassifier(
    criterion='gini',      # o 'entropy' para información
    max_depth=10,          # Profundidad máxima del árbol
    min_samples_split=5,   # Mínimo de muestras para dividir un nodo
    min_samples_leaf=2,    # Mínimo de muestras en hojas
    random_state=42
)

In [32]:
# Nota: Los árboles NO necesitan normalización, pero usar datos normalizados no afecta
dt_model.fit(X_train_scaled, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,10
,min_samples_split,5
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [33]:
# 2. Hacer predicciones
y_pred_dt = dt_model.predict(X_test_scaled)

## 3. Comparar desempeño entre ambos modelos

In [34]:
# SVM
accuracy = accuracy_score(y_test, y_pred)
print(f"\n{'='*50}")
print(f"ACCURACY SVM: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"{'='*50}")


ACCURACY SVM: 0.7083 (70.83%)


In [35]:
# Decision Tree
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"\n{'='*50}")
print(f"ACCURACY ÁRBOL DE DECISIÓN: {accuracy_dt:.4f} ({accuracy_dt*100:.2f}%)")
print(f"{'='*50}")


ACCURACY ÁRBOL DE DECISIÓN: 0.6667 (66.67%)


Se puede observar que el support vector machine entrenó mejor al modelo con una diferencia de 4% por sobre el decision tree

## 4. Realizar validación cruzada k-fold

In [36]:
# 1. Validación cruzada simple (solo accuracy)
print("="*60)
print("VALIDACIÓN CRUZADA - SVM")
print("="*60)

cv_scores_svm = cross_val_score(
    svm_model, 
    X_train_scaled, 
    y_train, 
    cv=5,  # 5 folds
    scoring='accuracy'
)

print(f"\nScores por fold: {cv_scores_svm}")
print(f"Accuracy promedio: {cv_scores_svm.mean():.4f} (+/- {cv_scores_svm.std() * 2:.4f})")
print(f"Mínimo: {cv_scores_svm.min():.4f}")
print(f"Máximo: {cv_scores_svm.max():.4f}")

VALIDACIÓN CRUZADA - SVM

Scores por fold: [0.9        0.89473684 0.73684211 0.73684211 0.73684211]
Accuracy promedio: 0.8011 (+/- 0.1573)
Mínimo: 0.7368
Máximo: 0.9000


In [37]:
# 2. Validación cruzada - Árbol de Decisión
print("\n" + "="*60)
print("VALIDACIÓN CRUZADA - ÁRBOL DE DECISIÓN")
print("="*60)

cv_scores_dt = cross_val_score(
    dt_model, 
    X_train_scaled, 
    y_train, 
    cv=5,
    scoring='accuracy'
)

print(f"\nScores por fold: {cv_scores_dt}")
print(f"Accuracy promedio: {cv_scores_dt.mean():.4f} (+/- {cv_scores_dt.std() * 2:.4f})")
print(f"Mínimo: {cv_scores_dt.min():.4f}")
print(f"Máximo: {cv_scores_dt.max():.4f}")


VALIDACIÓN CRUZADA - ÁRBOL DE DECISIÓN

Scores por fold: [0.85       0.84210526 0.63157895 0.73684211 0.68421053]
Accuracy promedio: 0.7489 (+/- 0.1721)
Mínimo: 0.6316
Máximo: 0.8500
